# Base study

This file contains code to run the evaluation for the base study for all instances

Evaluation runs:
- Baseline with Perses & Vulcan
- SeRu with Perses & Vulcan

## Setup

In [2]:
import java.io.File
import kotlin.io.path.Path
import kotlin.io.path.listDirectoryEntries
import kotlin.io.path.walk
import java.io.IOException
import java.util.concurrent.TimeUnit

val ignoredIssues = listOf("issue_2", "issue_2490")
val instances = File("instances")
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .filter { ignoredIssues.all { issue -> !it.parent.endsWith(issue) } }
    .map { it.toPath() }

data class CommandOutput(val stdout: String, val stderr: String)

fun String.runCommand(workingDir: File): CommandOutput? {
    return try {
        ProcessBuilder(*split(" ").toTypedArray())
            .directory(workingDir)
            .redirectOutput(ProcessBuilder.Redirect.PIPE)
            .redirectError(ProcessBuilder.Redirect.PIPE)
            .start()
            .also { it.waitFor(60, TimeUnit.MINUTES) }
            .let {
                CommandOutput(
                    stdout = it.inputStream.bufferedReader().readText(),
                    stderr = it.errorStream.bufferedReader().readText()
                )
            }
    } catch (e: IOException) {
        e.printStackTrace()
        null
    }
}

fun String.runCommandInRoot() = runCommand(File(".."))

## Compile SeRu

In [3]:
"go generate ./...".runCommandInRoot()
"go build".runCommandInRoot()

CommandOutput(stdout=, stderr=)

## Run SeRu

In [4]:
import java.nio.file.Path
import kotlin.io.path.pathString

// TODO use first cue file in folder instead of hardcoded "in.cue"
fun runSeruPersesIn(folder: Path) = "./seru -i ${folder.pathString}/in.cue -t ${folder.pathString}/test.sh -m".runCommandInRoot()
fun runSeruVulcanIn(folder: Path) = "./seru -i ${folder.pathString}/in.cue -t ${folder.pathString}/test.sh -m -r vulcan".runCommandInRoot()

In [6]:
import kotlin.io.path.*

val runsPerInstance = 5
val totalRuns = instances.toList().size * runsPerInstance

fun moveLatestSeruResultTo(dir: Path) = Path("..")
    .listDirectoryEntries(glob = "seru*")
    .maxBy { it.getLastModifiedTime() }
    .moveTo(dir / "seru_result")

fun saveLogsIfExisting(dir: Path, logs: CommandOutput?) = logs?.let {
    (dir / "log.txt").writeBytes(it.stdout.toByteArray())
    (dir / "err.txt").writeBytes(it.stderr.toByteArray())
}

fun runAllInstances(outputDir: String, runCmd: (Path) -> CommandOutput?) = instances
    .forEachIndexed { index, instance ->
        repeat(runsPerInstance) {
            val instanceDir = (Path(outputDir) / instance / "run_$it").createDirectories()
            val cur = it + (index * runsPerInstance)
            println("$cur/$totalRuns - Start run for $instanceDir")

            val output = runCmd(Path("study") / instance)

            moveLatestSeruResultTo(instanceDir)
            saveLogsIfExisting(instanceDir, output)
        }
    }

#### SeRu + Perses (all instances)

Takes around 40m (M1 MacBook Pro)

In [7]:
runAllInstances("results_perses", ::runSeruPersesIn)

0/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_perses/instances/extra/issue_2_inlined/final/run_0
1/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_1
2/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_2
3/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_3
4/80 - Start run for results_perses/instances/extra/issue_2_inlined/final/run_4
5/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_perses/instances/panic/issue_2584/v1/run_0
6/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_1
7/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_2
8/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_3
9/80 - Start run for results_perses/instances/panic/issue_2584/v1/run_4
10/80 - Start run for /Users/markus/Documents/UniProjects/MA/seru/study/results_perses/instances/panic/issue_2584/v2/run_0
11/80 - Start run

#### SeRu + Vulcan (all instances)

In [ ]:
runAllInstances("results_vulcan", ::runSeruVulcanIn)